<a href="https://colab.research.google.com/github/mshinko/Research-project/blob/main/Independent_Study2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=f9d84aaed31ccb1148cda41ab4ee687cd6a92278f76f9dd10581870b23ca13f6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import transformers

print(transformers.__version__)

4.33.2


In [3]:
model_checkpoint = "t5-small"

In [4]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 557.1 kB/s eta 0:00:00


In [5]:
pip install jsonlines

In [6]:
import jsonlines
import pandas as pd
from google.colab import files


uploaded = files.upload()
file_name = 'ta_assumptions'

# Read the JSONLines file and convert it into a DataFrame
data = []
with open(file_name, 'r', encoding='utf-8') as f:
    for item in jsonlines.Reader(f):
        data.append(item)

df = pd.DataFrame(data)


print(df.head())

Saving ta_assumptions to ta_assumptions
                                                text  ...  _timestamp
0  Implanted Coral into Experimental Tank- Prepar...  ...  1668447454
1  Recruited Two More Specialized Team Members- A...  ...  1668447486
2  Scheduled Meetings with Industry Advisors- Con...  ...  1668447509
3  Meet With, Contact & Gather Advisory Board Mem...  ...  1668447518
4  Review & Choose Awareness Campaign Product Kni...  ...  1668447571

[5 rows x 9 columns]


In [8]:
from datasets import load_dataset
import evaluate

metric = evaluate.load("rouge")

In [9]:
df

,text,_input_hash,_task_hash,options,_view_id,config,accept,answer,_timestamp
0,Implanted Coral into Experimental Tank- Prepar...,2142334788,1581322603,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption5P],accept,1668447454
1,Recruited Two More Specialized Team Members- A...,-669681360,1461625371,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},"[Assumption5T, Assumption5P]",accept,1668447486
2,Scheduled Meetings with Industry Advisors- Con...,146315610,1045997382,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption5T],accept,1668447509
3,"Meet With, Contact & Gather Advisory Board Mem...",-203240244,-408314981,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption5T],accept,1668447518
4,Review & Choose Awareness Campaign Product Kni...,1258027177,-600457207,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption3],accept,1668447571
...,...,...,...,...,...,...,...,...,...
262,Define Specific Week 2 Tasks Per Team Member- ...,1617565148,-615789806,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption5T],accept,1668450232
263,Conduct Elevator Pitch for Continued Recruitme...,-535465119,-980748290,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption5T],accept,1668450247
264,Unsure of Everyone's Availability- Need to dis...,1951361726,-1567760815,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption5T],accept,1668450251
265,Unsure of Time Needed to Brief/Introduce Team-...,1917032703,-2043216515,"[{'id': 'Assumtion1', 'text': 'Assumtion1'}, {...",choice,{'choice_style': 'multiple'},[Assumption5T],accept,1668450256


In [10]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
if model_checkpoint in ["google/t5-small-lm-adapt", "t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
  prefix = "summarize: "
else:
  prefix =""

In [15]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples["document"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["lables"] = labels["input_ids"]
    return model_inputs